In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# import warnings
# warnings.filterwarnings('ignore')

In [2]:
import xgboost as xgb
from xgboost import plot_importance
xgb.__version__

'0.90'

#### Load data

In [3]:
dataset = load_breast_cancer()
print(type(dataset))
print(dataset.keys())

<class 'sklearn.utils.Bunch'>
dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [4]:
X_features = dataset.data
y_label = dataset.target

print(X_features.shape)  # 569 instances, 30 features
print(y_label[:50])  # maligant 0, benign 1

(569, 30)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1]


In [5]:
cancer_df = pd.DataFrame(data=X_features, columns=dataset.feature_names)
cancer_df['target'] = y_label
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


#### Preprocess data

In [6]:
# generate train, test dataset
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size=0.2, random_state=0)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

#### Generate model

In [7]:
# specify parameters via map
param = {'max_depth':5, 'eta':1, 'objective':'binary:logistic' }
num_round = 10
print(f'param : {param} /// num_round : {num_round}')

# train model
bst = xgb.train(param, dtrain, num_round)

# make prediction
preds = bst.predict(dtest)

# evaluate
preds_convert = [1 if pred > 0.5 else 0 for pred in preds]
print('accuracy : ', len(y_test[y_test == preds_convert])/len(y_test))

param : {'max_depth': 5, 'eta': 1, 'objective': 'binary:logistic'} /// num_round : 10
accuracy :  0.9736842105263158


#### Generate model2

In [8]:
params = {
    'max_depth':3,
    'eta':0.1,  # learning rate
    'objective':'binary:logistic',
    'eval_metric':'logloss'
}
num_rounds = 1000  # no. of iteration of Boosting
wlist = [(dtrain, 'train'),(dtest, 'eval')]
xgb_model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist, early_stopping_rounds=100)

[0]	train-logloss:0.61167	eval-logloss:0.613695
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 100 rounds.
[1]	train-logloss:0.544351	eval-logloss:0.545425
[2]	train-logloss:0.486154	eval-logloss:0.489016
[3]	train-logloss:0.438088	eval-logloss:0.440352
[4]	train-logloss:0.394117	eval-logloss:0.399575
[5]	train-logloss:0.356264	eval-logloss:0.364347
[6]	train-logloss:0.325146	eval-logloss:0.332736
[7]	train-logloss:0.295618	eval-logloss:0.306005
[8]	train-logloss:0.269782	eval-logloss:0.282808
[9]	train-logloss:0.247249	eval-logloss:0.261858
[10]	train-logloss:0.227723	eval-logloss:0.243811
[11]	train-logloss:0.210154	eval-logloss:0.227002
[12]	train-logloss:0.194442	eval-logloss:0.213091
[13]	train-logloss:0.180529	eval-logloss:0.200189
[14]	train-logloss:0.166958	eval-logloss:0.186999
[15]	train-logloss:0.155558	eval-logloss:0.176615
[16]	train-logloss:0.144616	eval-logloss:0.167341
[17]	train-

[161]	train-logloss:0.007927	eval-logloss:0.048932
[162]	train-logloss:0.007887	eval-logloss:0.048575
[163]	train-logloss:0.007867	eval-logloss:0.048617
[164]	train-logloss:0.007838	eval-logloss:0.048751
[165]	train-logloss:0.007801	eval-logloss:0.048409
[166]	train-logloss:0.00778	eval-logloss:0.04845
[167]	train-logloss:0.007761	eval-logloss:0.048404
[168]	train-logloss:0.007741	eval-logloss:0.048245
[169]	train-logloss:0.007722	eval-logloss:0.048265
[170]	train-logloss:0.007703	eval-logloss:0.048311
[171]	train-logloss:0.007685	eval-logloss:0.048147
[172]	train-logloss:0.007666	eval-logloss:0.048265
[173]	train-logloss:0.007622	eval-logloss:0.048387
[174]	train-logloss:0.007604	eval-logloss:0.048226
[175]	train-logloss:0.007586	eval-logloss:0.048304
[176]	train-logloss:0.00756	eval-logloss:0.048426
[177]	train-logloss:0.007543	eval-logloss:0.048454
[178]	train-logloss:0.007495	eval-logloss:0.048619
[179]	train-logloss:0.007478	eval-logloss:0.048464
[180]	train-logloss:0.007461	eval-

[323]	train-logloss:0.00591	eval-logloss:0.045098
[324]	train-logloss:0.005903	eval-logloss:0.04502
[325]	train-logloss:0.005896	eval-logloss:0.045072
[326]	train-logloss:0.005888	eval-logloss:0.045092
[327]	train-logloss:0.005881	eval-logloss:0.045004
[328]	train-logloss:0.005874	eval-logloss:0.045043
[329]	train-logloss:0.005867	eval-logloss:0.045092
[330]	train-logloss:0.00586	eval-logloss:0.045146
[331]	train-logloss:0.005853	eval-logloss:0.044989
[332]	train-logloss:0.005846	eval-logloss:0.045048
[333]	train-logloss:0.005839	eval-logloss:0.044971
[334]	train-logloss:0.005832	eval-logloss:0.04481
[335]	train-logloss:0.005825	eval-logloss:0.044871
[336]	train-logloss:0.005818	eval-logloss:0.044909
[337]	train-logloss:0.005811	eval-logloss:0.044963
[338]	train-logloss:0.005804	eval-logloss:0.044807
[339]	train-logloss:0.005798	eval-logloss:0.044863
[340]	train-logloss:0.005791	eval-logloss:0.044847
[341]	train-logloss:0.005784	eval-logloss:0.044686
[342]	train-logloss:0.005777	eval-l

[485]	train-logloss:0.005055	eval-logloss:0.042435
[486]	train-logloss:0.005052	eval-logloss:0.042509
[487]	train-logloss:0.005048	eval-logloss:0.042454
[488]	train-logloss:0.005045	eval-logloss:0.042314
[489]	train-logloss:0.005042	eval-logloss:0.042347
[490]	train-logloss:0.005039	eval-logloss:0.042412
[491]	train-logloss:0.005035	eval-logloss:0.042315
[492]	train-logloss:0.005032	eval-logloss:0.042389
[493]	train-logloss:0.005029	eval-logloss:0.042295
[494]	train-logloss:0.005026	eval-logloss:0.042327
[495]	train-logloss:0.005023	eval-logloss:0.04219
[496]	train-logloss:0.00502	eval-logloss:0.042256
[497]	train-logloss:0.005016	eval-logloss:0.042202
[498]	train-logloss:0.005013	eval-logloss:0.042275
[499]	train-logloss:0.00501	eval-logloss:0.042206
[500]	train-logloss:0.005007	eval-logloss:0.042238
[501]	train-logloss:0.005004	eval-logloss:0.042227
[502]	train-logloss:0.005001	eval-logloss:0.042093
[503]	train-logloss:0.004998	eval-logloss:0.042166
[504]	train-logloss:0.004995	eval-

[647]	train-logloss:0.00469	eval-logloss:0.041412
[648]	train-logloss:0.00469	eval-logloss:0.041412
[649]	train-logloss:0.00469	eval-logloss:0.041412
[650]	train-logloss:0.00469	eval-logloss:0.041412
[651]	train-logloss:0.00469	eval-logloss:0.041412
[652]	train-logloss:0.00469	eval-logloss:0.041412
[653]	train-logloss:0.00469	eval-logloss:0.041412
[654]	train-logloss:0.00469	eval-logloss:0.041412
[655]	train-logloss:0.00469	eval-logloss:0.041412
[656]	train-logloss:0.00469	eval-logloss:0.041412
[657]	train-logloss:0.00469	eval-logloss:0.041412
[658]	train-logloss:0.00469	eval-logloss:0.041412
[659]	train-logloss:0.00469	eval-logloss:0.041412
[660]	train-logloss:0.00469	eval-logloss:0.041412
[661]	train-logloss:0.00469	eval-logloss:0.041412
[662]	train-logloss:0.00469	eval-logloss:0.041412
[663]	train-logloss:0.00469	eval-logloss:0.041412
[664]	train-logloss:0.00469	eval-logloss:0.041412
[665]	train-logloss:0.00469	eval-logloss:0.041412
[666]	train-logloss:0.00469	eval-logloss:0.041412


In [9]:
# make prediction
preds = xgb_model.predict(dtest)

# evaluate
preds_convert = [1 if pred > 0.5 else 0 for pred in preds]
print('accuracy : ', len(y_test[y_test == preds_convert])/len(y_test))

accuracy :  0.9824561403508771


#### Generate model3

In [10]:
params = {
    'max_depth':3,
    'eta':0.01,  # learning rate
    'objective':'binary:logistic',
    'eval_metric':'logloss'
}
num_rounds = 10000  # no. of iteration of Boosting
wlist = [(dtrain, 'train'),(dtest, 'eval')]
xgb_model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist, early_stopping_rounds=1000)

[0]	train-logloss:0.684628	eval-logloss:0.684844
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 1000 rounds.
[1]	train-logloss:0.676271	eval-logloss:0.676567
[2]	train-logloss:0.668072	eval-logloss:0.668446
[3]	train-logloss:0.660027	eval-logloss:0.660603
[4]	train-logloss:0.652131	eval-logloss:0.65278
[5]	train-logloss:0.64438	eval-logloss:0.645056
[6]	train-logloss:0.63677	eval-logloss:0.637638
[7]	train-logloss:0.629297	eval-logloss:0.630069
[8]	train-logloss:0.621959	eval-logloss:0.622916
[9]	train-logloss:0.614749	eval-logloss:0.615612
[10]	train-logloss:0.607668	eval-logloss:0.608595
[11]	train-logloss:0.60071	eval-logloss:0.60166
[12]	train-logloss:0.593873	eval-logloss:0.594886
[13]	train-logloss:0.587153	eval-logloss:0.588187
[14]	train-logloss:0.580549	eval-logloss:0.581754
[15]	train-logloss:0.574058	eval-logloss:0.575172
[16]	train-logloss:0.567634	eval-logloss:0.568648
[17]	train-log

[161]	train-logloss:0.155903	eval-logloss:0.177297
[162]	train-logloss:0.154758	eval-logloss:0.176186
[163]	train-logloss:0.153623	eval-logloss:0.175053
[164]	train-logloss:0.152502	eval-logloss:0.173884
[165]	train-logloss:0.151469	eval-logloss:0.17299
[166]	train-logloss:0.150392	eval-logloss:0.17193
[167]	train-logloss:0.1493	eval-logloss:0.170997
[168]	train-logloss:0.148275	eval-logloss:0.169977
[169]	train-logloss:0.147243	eval-logloss:0.169075
[170]	train-logloss:0.146239	eval-logloss:0.168078
[171]	train-logloss:0.145216	eval-logloss:0.167072
[172]	train-logloss:0.144177	eval-logloss:0.166221
[173]	train-logloss:0.143174	eval-logloss:0.165278
[174]	train-logloss:0.142219	eval-logloss:0.164295
[175]	train-logloss:0.141266	eval-logloss:0.163281
[176]	train-logloss:0.140268	eval-logloss:0.162283
[177]	train-logloss:0.139343	eval-logloss:0.16133
[178]	train-logloss:0.138419	eval-logloss:0.160412
[179]	train-logloss:0.137474	eval-logloss:0.159524
[180]	train-logloss:0.13651	eval-log

[323]	train-logloss:0.062123	eval-logloss:0.092411
[324]	train-logloss:0.06186	eval-logloss:0.092189
[325]	train-logloss:0.061578	eval-logloss:0.09191
[326]	train-logloss:0.0613	eval-logloss:0.091649
[327]	train-logloss:0.061018	eval-logloss:0.09138
[328]	train-logloss:0.060763	eval-logloss:0.091198
[329]	train-logloss:0.060492	eval-logloss:0.090827
[330]	train-logloss:0.060155	eval-logloss:0.090612
[331]	train-logloss:0.059888	eval-logloss:0.090273
[332]	train-logloss:0.059639	eval-logloss:0.090146
[333]	train-logloss:0.059377	eval-logloss:0.089846
[334]	train-logloss:0.059132	eval-logloss:0.089722
[335]	train-logloss:0.058874	eval-logloss:0.089473
[336]	train-logloss:0.058612	eval-logloss:0.089207
[337]	train-logloss:0.058356	eval-logloss:0.088933
[338]	train-logloss:0.0581	eval-logloss:0.08868
[339]	train-logloss:0.057848	eval-logloss:0.088335
[340]	train-logloss:0.057599	eval-logloss:0.088075
[341]	train-logloss:0.057366	eval-logloss:0.087901
[342]	train-logloss:0.057121	eval-loglo

[485]	train-logloss:0.033363	eval-logloss:0.066815
[486]	train-logloss:0.033264	eval-logloss:0.066752
[487]	train-logloss:0.033168	eval-logloss:0.066671
[488]	train-logloss:0.033069	eval-logloss:0.066571
[489]	train-logloss:0.032972	eval-logloss:0.066496
[490]	train-logloss:0.03284	eval-logloss:0.066447
[491]	train-logloss:0.032743	eval-logloss:0.066348
[492]	train-logloss:0.032647	eval-logloss:0.066275
[493]	train-logloss:0.032554	eval-logloss:0.066196
[494]	train-logloss:0.032425	eval-logloss:0.066157
[495]	train-logloss:0.03233	eval-logloss:0.06606
[496]	train-logloss:0.032237	eval-logloss:0.065988
[497]	train-logloss:0.032137	eval-logloss:0.065924
[498]	train-logloss:0.032038	eval-logloss:0.065862
[499]	train-logloss:0.031948	eval-logloss:0.065786
[500]	train-logloss:0.031856	eval-logloss:0.065716
[501]	train-logloss:0.031765	eval-logloss:0.065622
[502]	train-logloss:0.031644	eval-logloss:0.06566
[503]	train-logloss:0.031554	eval-logloss:0.065567
[504]	train-logloss:0.031464	eval-l

[647]	train-logloss:0.021219	eval-logloss:0.058492
[648]	train-logloss:0.021176	eval-logloss:0.0585
[649]	train-logloss:0.021124	eval-logloss:0.058516
[650]	train-logloss:0.021083	eval-logloss:0.058503
[651]	train-logloss:0.021033	eval-logloss:0.058393
[652]	train-logloss:0.020981	eval-logloss:0.058359
[653]	train-logloss:0.02093	eval-logloss:0.058376
[654]	train-logloss:0.020881	eval-logloss:0.058267
[655]	train-logloss:0.02084	eval-logloss:0.058276
[656]	train-logloss:0.020785	eval-logloss:0.05824
[657]	train-logloss:0.020734	eval-logloss:0.058239
[658]	train-logloss:0.020696	eval-logloss:0.058236
[659]	train-logloss:0.02065	eval-logloss:0.058239
[660]	train-logloss:0.020596	eval-logloss:0.058204
[661]	train-logloss:0.020548	eval-logloss:0.058097
[662]	train-logloss:0.020497	eval-logloss:0.058132
[663]	train-logloss:0.020444	eval-logloss:0.058097
[664]	train-logloss:0.020397	eval-logloss:0.057991
[665]	train-logloss:0.020361	eval-logloss:0.057989
[666]	train-logloss:0.020319	eval-log

...

[6536]	train-logloss:0.004701	eval-logloss:0.039677
[6537]	train-logloss:0.004701	eval-logloss:0.039682
[6538]	train-logloss:0.0047	eval-logloss:0.039683
[6539]	train-logloss:0.0047	eval-logloss:0.039691
[6540]	train-logloss:0.0047	eval-logloss:0.039687
[6541]	train-logloss:0.0047	eval-logloss:0.039688
[6542]	train-logloss:0.0047	eval-logloss:0.039694
[6543]	train-logloss:0.0047	eval-logloss:0.03969
[6544]	train-logloss:0.0047	eval-logloss:0.039698
[6545]	train-logloss:0.0047	eval-logloss:0.039694
[6546]	train-logloss:0.0047	eval-logloss:0.039695
[6547]	train-logloss:0.004699	eval-logloss:0.039703
[6548]	train-logloss:0.004699	eval-logloss:0.039699
[6549]	train-logloss:0.004699	eval-logloss:0.039704
[6550]	train-logloss:0.004699	eval-logloss:0.039706
[6551]	train-logloss:0.004699	eval-logloss:0.039702
[6552]	train-logloss:0.004699	eval-logloss:0.03971
[6553]	train-logloss:0.004699	eval-logloss:0.039711
[6554]	train-logloss:0.004699	eval-logloss:0.039716
[6555]	train-logloss:0.004698	ev

[6695]	train-logloss:0.004689	eval-logloss:0.040067
[6696]	train-logloss:0.004689	eval-logloss:0.040067
[6697]	train-logloss:0.004689	eval-logloss:0.040067
[6698]	train-logloss:0.004689	eval-logloss:0.040067
[6699]	train-logloss:0.004689	eval-logloss:0.040067
[6700]	train-logloss:0.004689	eval-logloss:0.040067
[6701]	train-logloss:0.004689	eval-logloss:0.040067
[6702]	train-logloss:0.004689	eval-logloss:0.040067
[6703]	train-logloss:0.004689	eval-logloss:0.040067
[6704]	train-logloss:0.004689	eval-logloss:0.040067
[6705]	train-logloss:0.004689	eval-logloss:0.040067
[6706]	train-logloss:0.004689	eval-logloss:0.040067
[6707]	train-logloss:0.004689	eval-logloss:0.040067
[6708]	train-logloss:0.004689	eval-logloss:0.040067
[6709]	train-logloss:0.004689	eval-logloss:0.040067
[6710]	train-logloss:0.004689	eval-logloss:0.040067
[6711]	train-logloss:0.004689	eval-logloss:0.040067
[6712]	train-logloss:0.004689	eval-logloss:0.040067
[6713]	train-logloss:0.004689	eval-logloss:0.040067
[6714]	train

[6853]	train-logloss:0.004689	eval-logloss:0.040067
[6854]	train-logloss:0.004689	eval-logloss:0.040067
[6855]	train-logloss:0.004689	eval-logloss:0.040067
[6856]	train-logloss:0.004689	eval-logloss:0.040067
[6857]	train-logloss:0.004689	eval-logloss:0.040067
[6858]	train-logloss:0.004689	eval-logloss:0.040067
[6859]	train-logloss:0.004689	eval-logloss:0.040067
[6860]	train-logloss:0.004689	eval-logloss:0.040067
[6861]	train-logloss:0.004689	eval-logloss:0.040067
[6862]	train-logloss:0.004689	eval-logloss:0.040067
[6863]	train-logloss:0.004689	eval-logloss:0.040067
[6864]	train-logloss:0.004689	eval-logloss:0.040067
[6865]	train-logloss:0.004689	eval-logloss:0.040067
[6866]	train-logloss:0.004689	eval-logloss:0.040067
[6867]	train-logloss:0.004689	eval-logloss:0.040067
[6868]	train-logloss:0.004689	eval-logloss:0.040067
[6869]	train-logloss:0.004689	eval-logloss:0.040067
[6870]	train-logloss:0.004689	eval-logloss:0.040067
[6871]	train-logloss:0.004689	eval-logloss:0.040067
[6872]	train

[7011]	train-logloss:0.004689	eval-logloss:0.040067
[7012]	train-logloss:0.004689	eval-logloss:0.040067
[7013]	train-logloss:0.004689	eval-logloss:0.040067
[7014]	train-logloss:0.004689	eval-logloss:0.040067
[7015]	train-logloss:0.004689	eval-logloss:0.040067
[7016]	train-logloss:0.004689	eval-logloss:0.040067
[7017]	train-logloss:0.004689	eval-logloss:0.040067
[7018]	train-logloss:0.004689	eval-logloss:0.040067
[7019]	train-logloss:0.004689	eval-logloss:0.040067
[7020]	train-logloss:0.004689	eval-logloss:0.040067
[7021]	train-logloss:0.004689	eval-logloss:0.040067
[7022]	train-logloss:0.004689	eval-logloss:0.040067
[7023]	train-logloss:0.004689	eval-logloss:0.040067
[7024]	train-logloss:0.004689	eval-logloss:0.040067
[7025]	train-logloss:0.004689	eval-logloss:0.040067
[7026]	train-logloss:0.004689	eval-logloss:0.040067
[7027]	train-logloss:0.004689	eval-logloss:0.040067
[7028]	train-logloss:0.004689	eval-logloss:0.040067
[7029]	train-logloss:0.004689	eval-logloss:0.040067
[7030]	train

[7169]	train-logloss:0.004689	eval-logloss:0.040067
[7170]	train-logloss:0.004689	eval-logloss:0.040067
[7171]	train-logloss:0.004689	eval-logloss:0.040067
[7172]	train-logloss:0.004689	eval-logloss:0.040067
[7173]	train-logloss:0.004689	eval-logloss:0.040067
[7174]	train-logloss:0.004689	eval-logloss:0.040067
[7175]	train-logloss:0.004689	eval-logloss:0.040067
[7176]	train-logloss:0.004689	eval-logloss:0.040067
[7177]	train-logloss:0.004689	eval-logloss:0.040067
[7178]	train-logloss:0.004689	eval-logloss:0.040067
[7179]	train-logloss:0.004689	eval-logloss:0.040067
[7180]	train-logloss:0.004689	eval-logloss:0.040067
[7181]	train-logloss:0.004689	eval-logloss:0.040067
[7182]	train-logloss:0.004689	eval-logloss:0.040067
[7183]	train-logloss:0.004689	eval-logloss:0.040067
[7184]	train-logloss:0.004689	eval-logloss:0.040067
[7185]	train-logloss:0.004689	eval-logloss:0.040067
[7186]	train-logloss:0.004689	eval-logloss:0.040067
[7187]	train-logloss:0.004689	eval-logloss:0.040067
[7188]	train

In [11]:
# make prediction
preds = xgb_model.predict(dtest)

# evaluate
preds_convert = [1 if pred > 0.5 else 0 for pred in preds]
print('accuracy : ', len(y_test[y_test == preds_convert])/len(y_test))

accuracy :  0.9824561403508771
